In [1]:
from azureml.core import Workspace
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice
from azureml.core.model import Model

In [2]:
import azureml.core
print(azureml.core.VERSION)

1.36.0


# Get workspace

In [ ]:
from azureml.core.workspace import Workspace

ws = Workspace(subscription_id="<your-subscription-id>",
               resource_group="<your-resource-group-name>",
               workspace_name="<your-ws-name>")
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# Register the model

In [4]:
from azureml.core.model import Model
model = Model.register(model_path = "movie.pkl", # this points to a local file
                       model_name = "movie", # this is the name the model is registered as
                       tags = {'area': "movies", 'type': "svd"},
                       description = "svd model to predict movie ratings of users",
                       workspace = ws)

print(model.name, model.description, model.version)

Registering model movie
movie svd model to predict movie ratings of users 1


In [ ]:
# Create the Environment

In [5]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies 

conda_deps = CondaDependencies.create(conda_packages=['numpy==1.17.0'], pip_packages=['azureml-defaults', 'inference-schema', 'surprise'])
myenv = Environment(name='myenv')
myenv.python.conda_dependencies = conda_deps

In [ ]:
# Create the InferenceConfig

In [13]:
from azureml.core.model import InferenceConfig

inf_config = InferenceConfig(entry_script='score.py', environment=myenv)

# Provision the AKS Cluster with SSL
This is a one time setup. You can reuse this cluster for multiple deployments after it has been created. If you delete the cluster or the resource group that contains it, then you would have to recreate it.

> Note that if you have an AzureML Data Scientist role, you will not have permission to create compute resources. Talk to your workspace or IT admin to create the compute targets described in this section, if they do not already exist.

See code snippet below. Check the documentation [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-secure-web-service) for more details

In [16]:
# Use the default configuration (can also provide parameters to customize)

provisioning_config = AksCompute.provisioning_configuration()
# Leaf domain label generates a name using the formula
#  "<leaf-domain-label>######.<azure-region>.cloudapp.azure.net"
#  where "######" is a random series of characters
provisioning_config.enable_ssl(leaf_domain_label = "contoso", overwrite_existing_domain = False)

aks_name = 'my-aks-ssl-2' 
# Create the cluster
aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_name, 
                                  provisioning_configuration = provisioning_config)


In [18]:
%%time
aks_target.wait_for_completion(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
None
CPU times: user 43.4 ms, sys: 3.9 ms, total: 47.3 ms
Wall time: 344 ms


# Deploy web service to AKS

In [ ]:
#deployment_config = AksWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, auth_enabled=False)

In [19]:
%%time

aks_config = AksWebservice.deploy_configuration()

aks_service_name ='aks-service-ssl-1'

aks_service = Model.deploy(workspace=ws,
                           name=aks_service_name,
                           models=[model],
                           inference_config=inf_config,
                           deployment_config=aks_config,
                           deployment_target=aks_target,
                          overwrite=True)

aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-12-16 08:08:00+00:00 Creating Container Registry if not exists.
2021-12-16 08:08:00+00:00 Registering the environment.
2021-12-16 08:08:01+00:00 Building image..
2021-12-16 08:14:27+00:00 Creating resources in AKS.
2021-12-16 08:14:28+00:00 Submitting deployment to compute.
2021-12-16 08:14:28+00:00 Checking the status of deployment aks-service-ssl-1..
2021-12-16 08:17:48+00:00 Checking the status of inference endpoint aks-service-ssl-1.
Succeeded
AKS service creation operation finished, operation "Succeeded"
Healthy
CPU times: user 1.45 s, sys: 137 ms, total: 1.59 s
Wall time: 9min 58s


# Test the web service using run method
We test the web sevice by passing data.
Run() method retrieves API keys behind the scenes to make sure that call is authenticated.

In [21]:
%%time
import json

input_payload = json.dumps({
    'user_id': "1",
    'movie_id': "1029"
})

output = aks_service.run(input_payload)

print(output)

[1, 1029, None, 2.6728017859274695, {'was_impossible': False}]
CPU times: user 3.38 ms, sys: 197 µs, total: 3.58 ms
Wall time: 12.3 ms
